## Download CMIP6 data from ESGF
A short sample script how to download the CMIP6 data needed for `Examples.ipynb`

There are multiple ways to search for and download data from ESGF (e.g. `esgf-pyclient`).<br>
For most of those you'll need to **register** with one the the ESGF data portals (e.g. https://esgf-node.llnl.gov/projects/esgf-llnl/)

The simplest way to download the data needed for `Examples.ipynb` is using `wget` (would not recommend if multiple different models and variables are needed):

In [2]:
import wget

Here we'd like to download the x- and y- velocities (uo and vo), temperatures (thetao), salinities (so) and vertical cell thicknesses (thkcello) for the CanESM5 model for 2001-2014:<br>
First we'll specify the path were to save files, **make sure to have enough storage space!**

In [3]:
path = 'YOUR_PATH/'

Further, we'll have to specify an available node and the data files to finally use wget to download the files:

In [4]:
var=['uo','vo','thetao','so']
for i in var:
    node = 'http://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/CCCma/CanESM5/historical/r1i1p1f1/Omon/'+i+'/gn/v20190429/' #we'll use the esgf-data1.llnl.gov node
    file = i+'_Omon_CanESM5_historical_r1i1p1f1_gn_200101-201012.nc' #uo, vo, thetao and so files for the CanESM5 model for 2001-2010
    a = node + file
    b = path + file
    print(a)
    print(b)
    print('downloading...')
    wget.download(a,b)

http://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/CCCma/CanESM5/historical/r1i1p1f1/Omon/uo/gn/v20190429/uo_Omon_CanESM5_historical_r1i1p1f1_gn_200101-201012.nc
/mnt/scratch/scratch/swinkelbauer/CMIP/uo_Omon_CanESM5_historical_r1i1p1f1_gn_200101-201012.nc
downloading...


KeyboardInterrupt: 

We further need the exact vertical cell extents (thkcello):

In [ ]:
node = 'http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/CCCma/CanESM5/historical/r1i1p1f1/Ofx/thkcello/gn/v20190429/' #we'll use the esgf-data1.llnl.gov node
file = 'thkcello_Ofx_CanESM5_historical_r1i1p1f1_gn.nc' #uo, vo, thetao and so files for the CanESM5 model for 2001-2010
a = node + file
b = path + file
print(a)
print(b)
print('downloading...')
wget.download(a,b)

**For downloading other files you can find the respective nodes and file names on the ESGF search interface (https://esgf-node.llnl.gov/search/cmip6/)**

## Download Reanalyses data from OpenDAP <br>
The last example uses GLORYS2V4 reanalysis data obtained via the Copernicus Marine Service.<br>
To download the data on the native grids via OpenDAP you'll have to register at: https://marine.copernicus.eu/ <br>
You may than proceed to download the needed data from the OpenDAP server (https://tds.mercator-ocean.fr/thredds3/glorys2v4/glorys2v4_dbb_02.html). <br>
The following script shows the download of the x-velocity (vozocrtx) component (**again make sure to have enough storage space!**):

In [1]:
import xarray as xa
import requests as rq
from datetime import date, timedelta
import logging
from requests.adapters import HTTPAdapter, Retry

In [2]:
session = rq.Session()
session.auth = ("YOUR_USERNAME","YOUR_PASSWORD")

In [3]:
print('get data u')
storeu = xa.backends.PydapDataStore.open('http://tds.mercator-ocean.fr/thredds3/dodsC/glorys2v4-monthly-gridU?nav_lon[0:1:1020][0:1:1441],nav_lat[0:1:1020][0:1:1441],deptht[0:1:74],time_counter[0:1:347],vozocrtx[0:1:347][0:1:74][0:1:1020][0:1:1441]',session = session)
print('open data')
datapgnu = xa.open_dataset(storeu,decode_times=False).isel(time_counter=slice(0,12))

get data u
open data


The time_counter is given as days since 1950-01-01 00:00:00, we can convert it using xarrays decode_cf:

In [4]:
datapgnu=xa.decode_cf(datapgnu)

Due to the size of the individual files we'll want to save monthly files:<br>
Note: we use the slice option to select the individual months in order to keep the time_counter dimension!

In [5]:
#Specify were to save the data:
path='YOUR_PATH/'

In [6]:
for j in range(len(datapgnu.time_counter)):
    print(datapgnu.time_counter[j])
    datapgnu.sel(time_counter=slice(datapgnu.time_counter[j],datapgnu.time_counter[j])).to_netcdf(path+'GLORYS2V4_uo_'+str(datapgnu.time_counter[0].dt.year.values)+'-'+str(datapgnu.time_counter[0].dt.month.values)+'.nc')

<xarray.DataArray 'time_counter' ()>
array('1993-01-15T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time_counter  datetime64[ns] 1993-01-15
Attributes:
    standard_name:  time
    long_name:      Time axis
    axis:           T
    _ChunkSizes:    512


PermissionError: [Errno 13] Permission denied: '/mnt/users/staff/swinkelbauer/PhD/calculations/CMIP/Plots/Test/YOUR_PATH/GLORYS2V4_uo_1993-1.nc'

You may download the y-velocities (vomecrty), temperatures (votemper) and salinities (vosaline) the same way, for the cell thicknesses we've created a `mesh_e3t_field.nc`, which you may download from zenodo:

In [60]:
# download the cell thickness data
import urllib.request
import shutil

url = 'https://zenodo.org/records/10376250/files/'
file_name_z = 'mesh_e3t_field.nc'
with urllib.request.urlopen(url + file_name_z) as response, open(file_name_z, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

z = xa.open_dataset(file_name_z)

In [61]:
z.to_netcdf(path+'mesh_e3t_field.nc')

<xarray.Dataset>
Dimensions:      (ncl1: 75, ncl2: 1021, ncl3: 1442)
Dimensions without coordinates: ncl1, ncl2, ncl3
Data variables:
    e3t_0_field  (ncl1, ncl2, ncl3) float32 ...